In [1]:
from neo4j import GraphDatabase
import pandas as pd
import os

## Folder Path

In [2]:
input_folder_path = "../../Input/csv_files/"

## Data Loading

In [3]:
df = pd.read_csv(os.path.abspath(os.path.join(input_folder_path, "sora_image_sample_1.csv")))

## Neo4j Config

In [4]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [5]:
config = load_neo4j_config("../Encryption/Neo4j-1cf46bb6-Created-2025-04-10.txt")

In [6]:
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")

## Driver Initialisation

In [7]:
# Create driver
driver = GraphDatabase.driver(uri, auth=(username, password))

## Test Connection

In [8]:
# Example query to test
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        print(result.single()["message"])

In [9]:
test_connection()

Connection successful!


## Upload Image

In [10]:
def upload_image(tx, filename, desc, url):
    tx.run("""
        MATCH (n) DETACH DELETE n
        MERGE (i:Image {filename: $filename})
        SET i.description = $desc, i.url = $url
    """, filename=filename, desc=desc, url=url)

In [11]:
print(df.columns.tolist())

['filename', 'description', 'url']


In [12]:
with driver.session() as session:
    for _, row in df.iterrows():
        session.execute_write(upload_image, row['filename'], row['description'], row['url'])

In [13]:
def view_images(tx):
    result = tx.run("MATCH (i:Image) RETURN i.filename, i.url")
    return [record.data() for record in result]

In [14]:
with driver.session() as session:
    for _, row in df.iterrows():
        session.execute_read(view_images)

In [15]:
import requests
from PIL import Image
from io import BytesIO

In [16]:
# def display_images(image_data):
#     for img in image_data:
#         print(f"Title: {img['i.filename']}")
#         print(f"Viewer URL: {img['i.url']}")

#         try:
#             headers = {
#                 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
#                 "Referer": img['i.url'],  # Referrer must match the page where the image is embedded
#                 "Accept": "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
#                 "Accept-Language": "en-US,en;q=0.9",
#                 "DNT": "1",
#                 "Connection": "keep-alive"
#             }

#             # Step 1: Get the Sora viewer page
#             response = requests.get(img['i.url'], headers=headers)
#             response.raise_for_status()

#             # Step 2: Parse <img> tag from HTML
#             soup = BeautifulSoup(response.content, "html.parser")
#             image_tag = soup.find("img")

#             if image_tag:
#                 # BeautifulSoup auto-unescapes &amp; -> &
#                 image_url = image_tag.get("src")
#                 print(f"✅ Found image URL: {image_url}")

#                 # Step 3: Fetch and display image
#                 image_response = requests.get(image_url, headers=headers)
#                 image_response.raise_for_status()

#                 if "image" in image_response.headers.get("Content-Type", ""):
#                     img_pil = Image.open(BytesIO(image_response.content))
#                     img_pil.show()
#                 else:
#                     print("❌ The fetched URL is not an image.")
#             else:
#                 print("❌ No <img> tag found in HTML.")
#         except Exception as e:
#             print(f"⚠️ Error: {e}")

In [17]:
def display_image_from_url(image_url):
    try:
        headers = {
            "User-Agent": "Mozilla/5.0"
        }
        response = requests.get(image_url, headers=headers)
        response.raise_for_status()

        if "image" in response.headers.get("Content-Type", ""):
            img = Image.open(BytesIO(response.content))
            img.show()
        else:
            print("❌ The content fetched is not an image.")
    except Exception as e:
        print("⚠️ Error fetching image:", e)

In [18]:
with driver.session() as session:
    images = session.execute_read(view_images)
    for img in images:
        url = img['i.url']
        display_image_from_url(url)